# Final Project CPSC 381: Introduction to Machine learning
Diego Aspinwall, Ethan Mathieu, Taimur Moolji

# 01: Data Loading + Cleaning

In [ ]:
pip install yfinance

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pickle

GREEN = "#56D6B3"
BLUE = "#636FFA"
RED = "#ED8375"

## 01-A: Loading Oil Data

In [ ]:
# Data load + formatting from the CSV
def load_WTI_data():
    path_name = "./Cushing_OK_WTI_Spot_Price_FOB_20200619.csv" # NOTE: Replace with your filepath
    df = pd.read_csv(path_name, header=4, parse_dates=[0])
    df.columns=["Date", "WTI"]
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    return df

WTI = load_WTI_data()

In [ ]:
# Data load + formatting from the CSV
def load_Brent_data():
    path_name = "./BrentOilPrices.csv" # NOTE: Replace with your filepath
    df = pd.read_csv(path_name, parse_dates=[0])
    df.columns=["Date", "Brent"]
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    return df

brent = load_Brent_data()


In [ ]:
# Combine the two price series into a single dataframe
oil_data = pd.merge(WTI, brent, left_index=True, right_index=True, how= "inner")
oil_data["Spread"] = oil_data["WTI"] - oil_data["Brent"]
oil_data["WTI_Daily_Change"] = oil_data["WTI"].pct_change()
oil_data["Brent_Daily_Change"] = oil_data["Brent"].pct_change()


### 01-A1: EDA of Oil Prices and Spread over Time

In [ ]:
# Graph the price of each of the oil types, as well as the premium of WTI to Brent
fig, axes = plt.subplots(1, 2, figsize=(15, 4))  
axes = axes.flatten()

axes[0].plot(oil_data.index, oil_data["WTI"], label = "WTI", color = GREEN)
axes[0].plot(oil_data.index, oil_data["Brent"], label = "Brent", color = BLUE)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('WTI Price')
axes[0].set_title('Oil Prices over Time')
axes[0].legend()

data_sub = oil_data[oil_data["Spread"] > -30]
color_seq = list(data_sub["Spread"].apply(lambda x: RED if x<0 else GREEN))
axes[1].bar(data_sub.index, data_sub["Spread"], color = color_seq)
axes[1].set_xlabel("Date")
axes[1].set_ylabel("Spread")
axes[1].set_title('WTI Premium to Brent over Time')

fig.suptitle("Oil Prices and Spread over Time")

plt.show()

### 01-A2: EDA of 1-Day Percent Change in Oil Prices

In [ ]:
# Below we plot histograms of the 1 day percent change in price of Brent and WTI

fig, axes = plt.subplots(1, 2, figsize=(15, 4))  
axes = axes.flatten()

axes[0].hist( oil_data["Brent_Daily_Change"]*100, color = BLUE, bins = 100)
axes[0].set_xlabel('Daily % Change in Brent Price')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of 1 Day Change in Brent Price')

axes[1].hist( oil_data["WTI_Daily_Change"]*100, color = GREEN, bins = 100)
axes[1].set_xlabel('Daily % Change in WTI Price')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of 1 Day Change in WTI Price')

fig.suptitle("Distribution of 1 Day % Change in Oil Prices")

plt.show()

## 01-B: Loading Price Data

In [ ]:
# Pull Stock Data for the following: 

# Exxon Mobil - NYSE: XOM
# Chevron - NYSE: CVX
# Shell - NYSE: SHEL
# Total Energies - NYSE: TTE
# ConocoPhillips - NYSE: COP
# BP - NYSE: BP
# Equinor ASA - NYSE: EQNR
# Canadian Natural Resources - NYSE: CNQ
# Marathon Petroleum - NYSE: MPC


# IMPORTANT: Keep this set to false unless the data file is corrupted or you dont have it. YF API limits are super stringent
REFRESH_PRICE_DATA_PULL = False
REFRESH_SHARE_DATA_PULL = False
tickers = ["XOM", 'CVX', 'COP', 'BP', 'EQNR', 'CNQ', 'MPC']
tickers_to_names = {"XOM" : "Exxon Mobil",
                    'CVX' : "Chevron",
                    'COP' : "Conoco Phillips",
                    'BP' : "British Petroleum",
                    'EQNR':  "Equinor ASA",
                    'CNQ' : "Canadian Natural Resources",
                    'MPC' : "Marathon Petroleum"}

In [ ]:
# Toggle pulling the data again with the flag in the previous cell
if REFRESH_PRICE_DATA_PULL == True:

    # Use YF Database to pull stock data
    def get_price_data(tickers, start_date, end_date):
        """
        Get price time series data for each ticker between specified dates.

        Parameters:
            tickers (list): List of ticker symbols.
            start_date (str): Start date in 'YYYY-MM-DD' format.
            end_date (str): End date in 'YYYY-MM-DD' format.

        Returns:
            dict: Dictionary containing price data for each ticker.
        """
        price_data = {}
        for ticker in tickers:
            try:
                data = yf.download(ticker, start=start_date, end=end_date)
                price_data[ticker] = data
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
        return price_data


    # Pull stock data for specified tickers
    START_DATE = '2015-11-04'
    END_DATE = '2020-06-19'

    # This returns a dictionary that has tickername: Dataframe of stock prices + various other data
    price_data = get_price_data(tickers, START_DATE, END_DATE)

    
    def save_price_data(price_data, filename):
        """
        Save price data dictionary to a file using pickle.

        Parameters:
            price_data (dict): Dictionary containing price data for each ticker.
            filename (str): Name of the file to save.
        """
        with open(filename, 'wb') as f:
            pickle.dump(price_data, f)

    # Saves the dictionary to a pkl file for easy data pull without hitting API refresh limit
    save_price_data(price_data, 'price_data.pkl')


In [ ]:
# Toggle pulling the data again with the flag in the previous cell
if REFRESH_SHARE_DATA_PULL == True:
    # Use YF Database to pull stock data
    def get_share_data(tickers, start_date, end_date):
        """
        Get price time series data for each ticker between specified dates.

        Parameters:
            tickers (list): List of ticker symbols.
            start_date (str): Start date in 'YYYY-MM-DD' format.
            end_date (str): End date in 'YYYY-MM-DD' format.

        Returns:
            dict: Dictionary containing price data for each ticker.
        """
        price_data = {}
        for ticker in tickers:
            try:
                tkr = yf.Ticker(ticker)
                data = tkr.get_shares_full(start=start_date, end=end_date)
                price_data[ticker] = data
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
        return price_data


    # Pull stock data for specified tickers
    start_date = '2015-11-04'
    end_date = '2020-06-19'

    # This returns a dictionary that has tickername: Dataframe of stock prices + various other data
    share_data = get_share_data(tickers, start_date, end_date)

    # Saves the dictionary to a pkl file for easy data pull without hitting API refresh limit
    save_price_data(share_data, 'share_data.pkl')

In [ ]:
def load_data_from_file(filename):
    """
    Load price data dictionary from a file using pickle.

    Parameters:
        filename (str): Name of the file to load.

    Returns:
        dict: Dictionary containing price data for each ticker.
    """
    with open(filename, 'rb') as f:
        return pickle.load(f)


# Pull Data from local file

path_to_price_data = 'price_data.pkl'
price_data = load_data_from_file(path_to_price_data)

path_to_share_data = 'share_data.pkl'
share_data = load_data_from_file(path_to_share_data)

# clean up share_data + price_data

START_DATE = '2015-11-04'
END_DATE = '2020-06-19'

full_date_range =  pd.date_range(start=START_DATE, end=END_DATE, freq='D')

for ticker in tickers:
    if not isinstance(share_data[ticker], pd.Series):
        share_data[ticker] = pd.Series(share_data[ticker]["Share Count"])

    if share_data[ticker].index.duplicated().any():
        share_data[ticker] = share_data[ticker][~share_data[ticker].index.duplicated(keep='last')]
    
    share_data[ticker] = share_data[ticker].tz_localize(None)
    share_data[ticker] = share_data[ticker].reindex(full_date_range)
    share_data[ticker] = share_data[ticker].ffill()
    share_data[ticker] = share_data[ticker].bfill()
    
    price_data[ticker] = price_data[ticker].tz_localize(None)
    price_data[ticker] = price_data[ticker].reindex(full_date_range)
    price_data[ticker] = price_data[ticker].ffill()
    price_data[ticker] = price_data[ticker].bfill()

    #additions

    # print(price_data[ticker])
    


### O1-B1: EDA of Price Time Series and Return Distributions

In [ ]:
for tkr in tickers:
    data = price_data[tkr]
    data = data[data.index > "2004-01-01"]
    data = data.sort_index()
    data["Daily Return"] = data["Adj Close"].pct_change()

    fig, ax = plt.subplots(1, 2, figsize = (15, 4))
    ax = ax.flatten()

    ax[0].plot(data.index, data["Adj Close"])
    ax[0].set_title(f"{tickers_to_names[tkr]} Stock Price History")
    ax[0].set_xlabel(f"Date")
    ax[0].set_ylabel(f"{tickers_to_names[tkr]} Price (USD)")

    ax[1].hist( data["Daily Return"] * 100, bins = 100)
    ax[1].set_title(f"{tickers_to_names[tkr]} Distribution of Daily Returns")
    ax[1].set_xlabel(f"{tickers_to_names[tkr]} Return (%)")
    ax[1].set_ylabel(f"Frequency")

    fig.suptitle(f"{tickers_to_names[tkr]} Price and Return History")


    


# 02: Construct Index

In [ ]:
# index_price_data = {"Date" : price_data["XOM"].index}

# for ticker in tickers:
#     index_price_data[f"{ticker} Price"] = price_data[ticker]["Adj Close"]

# index_share_data = {"Date" : share_data["XOM"].index}
# for ticker in tickers:
#     index_share_data[f"{ticker} Price"] = share_data[ticker]

# print(price_data["CVX"])

# for ticker in share_data:
#     share_data[ticker] = pd.DataFrame(list(share_data[ticker].items()), columns=['Date', 'Share Count'])
#     share_data[ticker].set_index('Date', inplace=True)
#     share_data[ticker].index = share_data[ticker].index.normalize()  # Normalize the date index

# share_data["XOM"] = pd.Series(share_data["XOM"]["Share Count"])
# print(type(share_data["XOM"]))
# print(type(share_data["XOM"]))
# share_data["XOM"] = share_data["XOM"].tz_localize(None)
# print(share_data["XOM"])
# s = pd.Series((share_data["XOM"]))
# s = s.tz_localize(None)
# print(s)

columns = []
for ticker in tickers_to_names.keys():
    columns.extend([
        f"{ticker} Price",
        f"{ticker} Share Count",
        f"{ticker} Market Cap"
    ])

columns.extend(["Index", "Total Market Cap"])

START_DATE = '2015-11-04'
END_DATE = '2020-06-19'

data = {col: [None] * len(pd.date_range(start=START_DATE, end=END_DATE)) for col in columns}

index_df = pd.DataFrame(data)
dates = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
index_df.index = dates

for date in dates:
    for ticker in tickers:
        index_df.at[date, f"{ticker} Price"] = price_data[ticker]["Adj Close"][date]
        index_df.at[date, f"{ticker} Share Count"] = share_data[ticker][date]
        index_df.at[date, f"{ticker} Market Cap"] = share_data[ticker][date] * price_data[ticker]["Adj Close"][date]
    
    index_df["Total Market Cap"] = sum(index_df[f"{ticker} Market Cap"] for ticker in tickers)
    index_df["Index"] = sum((index_df[f"{ticker} Market Cap"] / index_df["Total Market Cap"]) * index_df[f"{ticker} Price"] for ticker in tickers)

index_df

# 03: Fitting Model for Whole Index

# 04: Fitting Models for Single Stocks

In [ ]:
oil_data = oil_data[oil_data["Brent_Daily_Change"] > -1]
oil_data = oil_data[oil_data["WTI_Daily_Change"] > -1]

plt.scatter(oil_data["WTI_Daily_Change"], oil_data["Brent_Daily_Change"])
plt.title("Daily % Change in Brent vs. Daily % Change in WTI")
plt.xlabel("Daily % Change in WTI")
plt.xlabel("Daily % Change in Brent")
plt.axhline(0, color='grey', linewidth=0.5, linestyle='--')  
plt.axvline(0, color='grey', linewidth=0.5, linestyle='--') 
plt.show()


In [ ]:
for tkr in tickers:
    data = price_data[tkr]
    data = data[data.index > "2004-01-01"]

    data = data.merge(oil_data, left_index=True, right_index=True, how= "inner")
    data = data[data["WTI_Daily_Change"] > -1]

    data = data.sort_index()
    data["Daily Return"] = data["Adj Close"].pct_change()


    fig, ax = plt.subplots(1, 2, figsize = (15, 4))
    ax = ax.flatten()

    ax[0].scatter(data["WTI_Daily_Change"], data["Daily Return"])
    ax[0].set_title(f"{tickers_to_names[tkr]} Scatterplot of WTI % Change vs. Stock REturn")
    ax[0].set_xlabel(f"WTI % Change")
    ax[0].set_ylabel(f"{tickers_to_names[tkr]} Return (%)")
    ax[0].axhline(0, color='grey', linewidth=0.5, linestyle='--')  
    ax[0].axvline(0, color='grey', linewidth=0.5, linestyle='--') 
    
    ax[1].scatter(data["Brent_Daily_Change"], data["Daily Return"])
    ax[1].set_title(f"{tickers_to_names[tkr]} Scatterplot of Brent % Change vs. Stock REturn")
    ax[1].set_xlabel(f"Brent % Change")
    ax[1].set_ylabel(f"{tickers_to_names[tkr]} Return (%)")
    ax[1].axhline(0, color='grey', linewidth=0.5, linestyle='--')  
    ax[1].axvline(0, color='grey', linewidth=0.5, linestyle='--')  

    fig.suptitle(f"{tickers_to_names[tkr]} Price and Return History")

# 05: Model Evaluation + Time Series Cross Validation